In [1]:
import pandas as pd
import torch
from torch.utils import data
from sklearn.model_selection import train_test_split

df = pd.read_csv('data.csv', delimiter=",")
print(df.columns) 

df = df.drop(['instant', 'dteday', 'casual', 'registered'], axis=1)

# print(df.describe())

# Split the data into train and test sets
train, test = train_test_split(df, test_size=0.2, random_state=42)

# print(train)

train_dataset = data.TensorDataset(torch.tensor(train.values[:,:-1]), torch.tensor(train.values[:,-1]))
test_dataset = data.TensorDataset(torch.tensor(test.values[:,:-1]), torch.tensor(test.values[:,-1]))
train_data_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=False)
test_data_loader = data.DataLoader(test_dataset, batch_size=128, shuffle=False, drop_last=False)

# print(train_dataset)

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')


In [2]:
import torch.nn as nn

class Evaluator(nn.Module):

    def __init__(self, num_inputs, num_hidden, num_outputs):
        super().__init__()
        # Initialize the modules we need to build the network
        self.linear1 = nn.Linear(num_inputs, num_hidden)
        self.batch_norm = nn.BatchNorm1d(num_hidden)
        self.act_fn = nn.LeakyReLU()
        self.linear2 = nn.Linear(num_hidden, num_hidden)
        self.batch_norm2 = nn.BatchNorm1d(num_hidden)
        self.linear3 = nn.Linear(num_hidden, num_outputs)

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        x = self.linear1(x)
        x = self.batch_norm(x)
        x = self.act_fn(x)
        x = self.linear2(x)
        x = self.batch_norm2(x)
        x = self.act_fn(x)
        x = self.linear3(x)
        return x

In [3]:
model = Evaluator(num_inputs=12, num_hidden=64, num_outputs=1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Evaluator(
  (linear1): Linear(in_features=12, out_features=64, bias=True)
  (batch_norm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): LeakyReLU(negative_slope=0.01)
  (linear2): Linear(in_features=64, out_features=64, bias=True)
  (batch_norm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=64, out_features=1, bias=True)
)

In [4]:
import numpy as np

In [5]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# def rmsle(y_true,y_pred):
#     msle = torch.mean(torch.tensor((torch.log(torch.clamp(y_pred,min=0) + 1) - torch.log(y_true + 1)) ** 2.0))
#     return torch.sqrt(msle)

loss_module = nn.MSELoss()

model.train() 

# Training loop
for epoch in range(100):
    for data_inputs, data_labels in train_data_loader:

        ## Step 1: Move input data to device (only strictly necessary if we use GPU)
        data_inputs = data_inputs.float().to(device)
        data_labels = data_labels.float().to(device)

        ## Step 2: Run the model on the input data
        preds = model(data_inputs)
        preds = preds.squeeze(dim=1) # Output is [Batch size, 1], but we want [Batch size]

        ## Step 3: Calculate the loss
        loss = loss_module(preds, data_labels)

        ## Step 4: Perform backpropagation
        # Before calculating the gradients, we need to ensure that they are all zero. 
        # The gradients would not be overwritten, but actually added to the existing ones.
        optimizer.zero_grad() 
        # Perform backpropagation
        loss.backward()

        ## Step 5: Update the parameters
        optimizer.step()
    print(f"Epoch: {epoch}, loss: {loss.item():.3}")



Epoch: 0, loss: 5.03e+04
Epoch: 1, loss: 4.85e+04
Epoch: 2, loss: 4.59e+04
Epoch: 3, loss: 4.22e+04
Epoch: 4, loss: 3.8e+04
Epoch: 5, loss: 3.34e+04
Epoch: 6, loss: 2.88e+04
Epoch: 7, loss: 2.39e+04
Epoch: 8, loss: 2e+04
Epoch: 9, loss: 1.55e+04
Epoch: 10, loss: 1.18e+04
Epoch: 11, loss: 8.77e+03
Epoch: 12, loss: 6.1e+03
Epoch: 13, loss: 4.21e+03
Epoch: 14, loss: 2.77e+03
Epoch: 15, loss: 1.77e+03
Epoch: 16, loss: 1.08e+03
Epoch: 17, loss: 6.62e+02
Epoch: 18, loss: 4.22e+02
Epoch: 19, loss: 3.48e+02
Epoch: 20, loss: 2.51e+02
Epoch: 21, loss: 2.88e+02
Epoch: 22, loss: 2.64e+02
Epoch: 23, loss: 2.6e+02
Epoch: 24, loss: 2.82e+02
Epoch: 25, loss: 2.53e+02
Epoch: 26, loss: 2.4e+02
Epoch: 27, loss: 1.82e+02
Epoch: 28, loss: 1.63e+02
Epoch: 29, loss: 1.32e+02
Epoch: 30, loss: 1e+02
Epoch: 31, loss: 70.8
Epoch: 32, loss: 56.6
Epoch: 33, loss: 64.8
Epoch: 34, loss: 49.5
Epoch: 35, loss: 46.5
Epoch: 36, loss: 32.7
Epoch: 37, loss: 42.8
Epoch: 38, loss: 41.7
Epoch: 39, loss: 34.6
Epoch: 40, loss:

In [18]:
targets = pd.read_csv("evaluation_data.csv",header=None)
targets = targets.drop([0], axis=1)
targets = targets.apply(pd.to_numeric, errors='coerce')
targets = targets.dropna()
print(targets)
inputs = torch.tensor(targets.values, dtype=torch.float32)
inputs = inputs.to(device)
# targets_dataset = data.TensorDataset(torch.tensor(targets.values, dtype=torch.float32))
# targets_data_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=False, drop_last=False)

       1    2     3     4    5    6    7    8     9       10    11      12
1     1.0  0.0   1.0   0.0  0.0  4.0  1.0  1.0  0.26  0.2273  0.56  0.3881
2     1.0  0.0   1.0   1.0  0.0  4.0  1.0  1.0  0.26  0.2727  0.56  0.0000
3     1.0  0.0   1.0   2.0  0.0  4.0  1.0  1.0  0.26  0.2727  0.56  0.0000
4     1.0  0.0   1.0   3.0  0.0  4.0  1.0  1.0  0.26  0.2576  0.56  0.1642
5     1.0  0.0   1.0   4.0  0.0  4.0  1.0  1.0  0.26  0.2576  0.56  0.1642
...   ...  ...   ...   ...  ...  ...  ...  ...   ...     ...   ...     ...
6489  1.0  1.0  12.0  19.0  0.0  1.0  1.0  2.0  0.26  0.2576  0.60  0.1642
6490  1.0  1.0  12.0  20.0  0.0  1.0  1.0  2.0  0.26  0.2576  0.60  0.1642
6491  1.0  1.0  12.0  21.0  0.0  1.0  1.0  1.0  0.26  0.2576  0.60  0.1642
6492  1.0  1.0  12.0  22.0  0.0  1.0  1.0  1.0  0.26  0.2727  0.56  0.1343
6493  1.0  1.0  12.0  23.0  0.0  1.0  1.0  1.0  0.26  0.2727  0.65  0.1343

[6493 rows x 12 columns]


In [19]:
def rmsle(y_true,y_pred):
    n = len(y_true)
    msle = np.mean([(np.log(max(y_pred[i],0) + 1) - np.log(y_true[i] + 1)) ** 2.0 for i in range(n)])
    return np.sqrt(msle)

model.eval() # Set model to eval mode

# all_preds = []

# with torch.no_grad():  # Disable gradient calculation for inference
#     for data_inputs in targets_dataset:
#         data_inputs = data_inputs[0].to(device)  # Move input data to device
#         preds = model(data_inputs.float())  # Perform forward pass
#         preds = preds.squeeze(dim=1)  # Output is [Batch size, 1], but we want [Batch size]
#         all_preds.append(preds.cpu().numpy())  # Move predictions to CPU and convert to numpy

with torch.no_grad():  # Disable gradient calculation for inference
    outputs = model(inputs).squeeze()

# print(all_preds)
# Concatenate all predictions
# all_preds = np.concatenate(all_preds)
# print(all_preds)

pd.DataFrame(outputs.detach().cpu()).to_csv("results.csv", index=False, header=None)

# pd.DataFrame(preds).to_csv("results.csv", index=False, header=None)
# preds = pd.read_csv("results.csv",header=None)
# print(rmsle(preds.numpy(), targets.values))